In [166]:
import pandas as pd
from tqdm import tqdm
from datetime import datetime
import json

# Récupérer toutes les gares de France

In [167]:
dfStop = pd.read_csv('../data_sncf/stops.txt')
dfStop.shape

(9176, 9)

In [168]:
dfStop = dfStop[dfStop['stop_id'].str.contains('StopPoint:OCETrain')]
dfStop.shape

(2447, 9)

In [169]:
allGare = dfStop['stop_name'].unique()
allGare.shape
print(allGare)

['Gare de Paris-St-Lazare' 'Gare de Rouen-Rive-Droite' 'Gare de Montville'
 ... 'Gare de Novillars' 'Gare de Deluz' 'Gare de Laissey']


# Extractions des voisins

In [170]:
dfStopTimes = pd.read_csv('../data_sncf/stop_times.txt', sep=",")
dfStopTimes.shape

/var/folders/0l/4rhc85qd4kl41w3sd26dml3h0000gn/T/ipykernel_50898/2347447891.py:1: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  dfStopTimes = pd.read_csv('../data_sncf/stop_times.txt', sep=",")


(209129, 9)

In [171]:
def str_to_time(time: str):
    time_split = time.split(':')
    if time_split[0] > '23':
        time_split[0] = str(int(time_split[0]) - 24)
    time = f'{time_split[0]}:{time_split[1]}:{time_split[2]}'
    time_object = datetime.strptime(time, '%H:%M:%S')
    return time_object

# Clean dataframe before calculating

In [172]:
# dfStopTimes['departure_time'] = dfStopTimes['departure_time'].apply(str_to_time)
# dfStopTimes.head()

In [173]:
def convert_str_to_time(toConvertTime: str):
    time_split = toConvertTime.split(':')
    hour = int(time_split[0])
    if hour > 23:
        hour -= 24
    time_object = datetime.strptime(f'{hour}:{time_split[1]}:{time_split[2]}', '%H:%M:%S')
    return time_object

In [174]:
def getVoisins(gare):
    # on recupere le stopID de notre gare
    stopID = dfStop[(dfStop.stop_name == gare)].stop_id.values[0]
    # on recupere les trajets qui passent par cette gare
    trajets = dfStopTimes[dfStopTimes['stop_id'] == stopID]
    listeVoisinsGare = {}

    for j in range(len(trajets)):
        # on recupere la position de la gare dans le trajet
        stopSequence = trajets['stop_sequence'].values[j]
        # on recupere le tripID du trajet
        tripId = trajets['trip_id'].values[j]
        # on recupere les times de notre gare
        departureTimeGare = convert_str_to_time(trajets['departure_time'].values[j])
        arivalTimeGare = convert_str_to_time(trajets['arrival_time'].values[j])
        try:
            # on recupere la gare précédente
            voisin1 = dfStopTimes[(dfStopTimes['trip_id'] == tripId) & (
                    dfStopTimes['stop_sequence'] == stopSequence - 1)]
            # on recupere le departure time de la gare précédente
            departureTimeVoisin1 = convert_str_to_time(voisin1.departure_time.values[0])
            # on recupere le nom de la gare
            voisin1 = dfStop[dfStop['stop_id'] ==
                             voisin1.stop_id.values[0]].stop_name.values[0]
            # on calcule le temps de trajet
            tempsTrajetvoisin1 = arivalTimeGare - departureTimeVoisin1
            # on ajoute le voisin et le temps de trajet dans le dictionnaire
            listeVoisinsGare[voisin1] = tempsTrajetvoisin1.seconds
        except:
            continue

        try:
            # on recupere la gare suivante
            voisin2 = dfStopTimes[(dfStopTimes['trip_id'] == tripId) & (
                    dfStopTimes['stop_sequence'] == stopSequence + 1)]
            # on recupere le departure time de la gare suivante
            arrivalTimeVoisin2 = convert_str_to_time(voisin2.departure_time.values[0])
            # on recupere le nom de la gare
            voisin2 = dfStop[dfStop['stop_id'] ==
                             voisin2.stop_id.values[0]].stop_name.values[0]
            # on calcule le temps de trajet
            tempsTrajetvoisin2 = arrivalTimeVoisin2 - departureTimeGare
            # on ajoute le voisin et le temps de trajet dans le dictionnaire
            listeVoisinsGare[voisin2] = tempsTrajetvoisin2.seconds
        except:
            continue

    # on retourne notre liste dans un set pour supprimer les doublons
    return listeVoisinsGare

In [175]:
graph = {}

for i in tqdm(range(2)):
    graph[allGare[i]] = getVoisins(allGare[i])


100%|██████████| 2/2 [00:16<00:00,  8.15s/it]


In [176]:
print(graph)

{'Gare de Paris-St-Lazare': {'Gare de Rouen-Rive-Droite': 5220, 'Gare de Vernon-Giverny': 3240, 'Gare de Mantes-la-Jolie': 2340, 'Gare de Caen': 7320, 'Gare de Evreux - Normandie': 5280, 'Gare de Lisieux': 6000}, 'Gare de Rouen-Rive-Droite': {'Gare de Paris-St-Lazare': 5220, 'Gare de Montville': 720, 'Gare de Yvetot': 1260, 'Gare de Oissel': 600, 'Gare de Le Havre': 2760, 'Gare de Elbeuf-St-Aubin': 960, 'Gare de Morgny': 900, 'Gare de Longuerue-Vieux-Manoir': 1080, 'Gare de Montérolier-Buchy': 1440, 'Gare de Barentin': 780, 'Gare de Sotteville': 360, 'Gare de Maromme': 420}}
